In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_boston
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error
from skimage.transform import resize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

dataset = np.loadtxt("tableft.csv", delimiter=',', usecols=range(2, 682))
np.random.shuffle(dataset)

# Separate the FLIM data (X) and average lifetime values (Y)
X = dataset[:, :679]
Y = dataset[:, 679]

# Split the data into training, validation, and test sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=42)

# Resize FLIM data to 32x32 using skimage's resize function
target_size = (32, 32)
X_train_resized = np.array([resize(sample, target_size) for sample in X_train])
X_train_resized = np.repeat(X_train_resized[..., np.newaxis], 3, -1)

X_val_resized = np.array([resize(sample, target_size) for sample in X_val])
X_val_resized = np.repeat(X_val_resized[..., np.newaxis], 3, -1)

X_test_resized = np.array([resize(sample, target_size) for sample in X_test])
X_test_resized = np.repeat(X_test_resized[..., np.newaxis], 3, -1)

# Load the pre-trained VGG-16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom layers on top of the VGG-16 base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='elu')(x)
x = Dense(4096, activation='elu')(x)
predictions = Dense(1, activation='linear')(x)

# Create the model with the custom layers on top of VGG-16
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers in the pre-trained VGG-16 base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='mae', optimizer=Adam(lr=0.001))

# Print the model summary
model.summary()

# Train the model with validation data
model.fit(X_train_resized, Y_train, batch_size=32, epochs=20, validation_data=(X_val_resized, Y_val), verbose=1)

# Evaluate the model on the training, validation, and test data
train_loss = model.evaluate(X_train_resized, Y_train, verbose=0)
val_loss = model.evaluate(X_val_resized, Y_val, verbose=0)
test_loss = model.evaluate(X_test_resized, Y_test, verbose=0)

# Calculate and print MAE, MSE, and RMSE for the training, validation, and test datasets
Y_train_pred = model.predict(X_train_resized)
Y_val_pred = model.predict(X_val_resized)
Y_test_pred = model.predict(X_test_resized)

train_mae = mean_absolute_error(Y_train, Y_train_pred)
val_mae = mean_absolute_error(Y_val, Y_val_pred)
test_mae = mean_absolute_error(Y_test, Y_test_pred)

train_mse = mean_squared_error(Y_train, Y_train_pred)
val_mse = mean_squared_error(Y_val, Y_val_pred)
test_mse = mean_squared_error(Y_test, Y_test_pred)

train_rmse = np.sqrt(train_mse)
val_rmse = np.sqrt(val_mse)
test_rmse = np.sqrt(test_mse)

print("Training Loss (MAE):", train_loss)
print("Validation Loss (MAE):", val_loss)
print("Test Loss (MAE):", test_loss)

print("Training MAE:", train_mae)
print("Validation MAE:", val_mae)
print("Test MAE:", test_mae)

print("Training MSE:", train_mse)
print("Validation MSE:", val_mse)
print("Test MSE:", test_mse)

print("Training RMSE:", train_rmse)
print("Validation RMSE:", val_rmse)
print("Test RMSE:", test_rmse)


2023-07-26 00:16:07.182254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-26 00:16:07.453783: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


58889256/58889256 [==============================] - 1s 0us/step


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

KeyboardInterrupt: 

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from skimage.transform import resize
from skopt import gp_minimize
from skopt.space import Real, Integer

# Load the FLIM data and average lifetime values from the CSV file
dataset = np.loadtxt("tableft.csv", delimiter=',', usecols=range(2, 682))
np.random.shuffle(dataset)

# Separate the FLIM data (X) and average lifetime values (Y)
X = dataset[:, :679]
Y = dataset[:, 679]

# Split the data into training, validation, and test sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size=0.5, random_state=42)

# Resize FLIM data to 32x32 using skimage's resize function and convert to 3 channels
target_size = (32, 32)
X_train_resized = np.array([resize(sample, target_size) for sample in X_train])
X_train_resized = np.repeat(X_train_resized[..., np.newaxis], 3, -1)

X_val_resized = np.array([resize(sample, target_size) for sample in X_val])
X_val_resized = np.repeat(X_val_resized[..., np.newaxis], 3, -1)

X_test_resized = np.array([resize(sample, target_size) for sample in X_test])
X_test_resized = np.repeat(X_test_resized[..., np.newaxis], 3, -1)

# Define the hyperparameter search space and evaluation function
space = [
    Real(1e-6, 1e-1, name='learning_rate', prior='log-uniform'),
    Integer(128, 2048, name='neurons_layer1'),
    Integer(128, 2048, name='neurons_layer2'),
    Integer(32, 256, name='batch_size')
]

# Define the evaluation function for skopt to optimize
def evaluate_model(params):
    learning_rate = params[0]
    neurons_layer1 = params[1]
    neurons_layer2 = params[2]
    batch_size = params[3]

    # Load the pre-trained VGG-16 model without the top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

    # Add custom layers on top of the VGG-16 base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(neurons_layer1, activation='relu')(x)
    x = Dense(neurons_layer2, activation='relu')(x)
    predictions = Dense(1, activation='linear')(x)

    # Create the model with the custom layers on top of VGG-16
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers in the pre-trained VGG-16 base model
    for layer in base_model.layers:
        layer.trainable = False

    # Compile the model with the specified learning rate
    model.compile(loss='mae', optimizer=Adam(learning_rate=learning_rate))

    # Train the model with the specified batch size
    model.fit(X_train_resized, Y_train, batch_size=batch_size, epochs=20, validation_data=(X_val_resized, Y_val), verbose=0)

    # Evaluate the model on the validation data
    val_loss = model.evaluate(X_val_resized, Y_val, verbose=0)
    
    return val_loss

# Optimize hyperparameters using gp_minimize from skopt
result = gp_minimize(evaluate_model, space, n_calls=20, random_state=42)

# Get the best hyperparameters found by skopt
best_learning_rate, best_neurons_layer1, best_neurons_layer2, best_batch_size = result.x

print("Best Learning Rate:", best_learning_rate)
print("Best Neurons in Layer 1:", best_neurons_layer1)
print("Best Neurons in Layer 2:", best_neurons_layer2)
print("Best Batch Size:", best_batch_size)

# Train the final model with the best hyperparameters on the full training set
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(loss='mae', optimizer=Adam(learning_rate=best_learning_rate))
model.fit(X_train_resized, Y_train, batch_size=best_batch_size, epochs=20, validation_data=(X_val_resized, Y_val), verbose=1)

# Evaluate the model on the test data using the best hyperparameters
test_loss = model.evaluate(X_test_resized, Y_test, verbose=0)

print("Test Loss (MAE):", test_loss)


2023-07-28 15:29:14.843083: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-28 15:29:15.064367: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


KeyboardInterrupt: 